<a href="https://colab.research.google.com/github/lrssv/TimeSeriesForecastingWindSun/blob/master/INMET_Dataset_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import timedelta
import itertools
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Recebe o arquivo csv
url= 'https://raw.githubusercontent.com/lrssv/TimeSeriesForecastingWindSun/master/inmet_pampulha_2018_2019.CSV'
df = pd.read_csv(url, sep=";", encoding = "ISO-8859-1", usecols=['Data','Hora','RADIACAO','VENTO_VEL']).fillna(0)

In [ ]:
#Refaz o formato da hora
df['Hora'] = df['Hora'].str.replace(r'\D', '')
df['Hora'] = [x[:-2] for x in df['Hora']]

In [ ]:
#Acrescenta um 0 a esquerda em VENTO_VEL e substitui ',' por '.'
df['VENTO_VEL'] = df['VENTO_VEL'].str.zfill(3)
df['VENTO_VEL'] = df['VENTO_VEL'].str.replace(',','.').fillna(0)

In [ ]:
#Substitui ',' por '.' em RADIACAO
df['RADIACAO'] = df['RADIACAO'].str.replace(',','.').fillna(0)

In [ ]:
#Substituir os valores negativos de radiacao
df['RADIACAO'] = [float(x) for x in df['RADIACAO']]
df = df.assign(RADIACAO = df.RADIACAO.where(df.RADIACAO.ge(0))).fillna(0)

In [ ]:
#Agrupa data+hora 
source_col_loc = df.columns.get_loc('Data') 
df['datetime'] = df.iloc[:,source_col_loc:source_col_loc+2].apply(
    lambda x: " ".join(x.astype(str)), axis=1)

In [ ]:
#Tranforma data+hora em Datetime e cria um novo dataframe
dataFormatada = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

d = {'date':dataFormatada, 'ventovel':df['VENTO_VEL'], 'radiacao':df['RADIACAO']}

dataFrame = pd.DataFrame(data=d)

frameList = list(dataFrame.date) #list of all dates in 'dataFrame'

In [ ]:
#Cria todas as datas existentes no banco de dados

serieStart = '2018-01-01 00:00:00' 
serieEnd = '2019-12-31 23:00:00	'

date = pd.date_range(start=serieStart, end=serieEnd, freq='1H')

dt = {'date': date}
frameDate = date #list of dates all dates in 'date'


In [ ]:
#Inclui datas faltantes e substitui o valor por nan

new_dates = []
new_values_ventovel = []
new_values_rad = []


for i in frameDate:
  if i in frameList:
    new_dates.append(i)
    new_values_ventovel.append(float(dataFrame[dataFrame['date']==i]['ventovel']))
    new_values_rad.append(float(dataFrame[dataFrame['date']==i]['radiacao']))

  else:
    new_dates.append(i)
    new_values_ventovel.append(np.nan)
    new_values_rad.append(np.nan)

In [ ]:
#Transforma em Serie e interpola os valores nan 
index = pd.DatetimeIndex(new_dates)
series_ventovel = pd.Series(new_values_ventovel, index=index).interpolate()
series_radiacao = pd.Series(new_values_rad, index=index).interpolate()

In [ ]:
#Exporta pra csv
series_ventovel.to_csv('/content/drive/My Drive/Data Files/series_ventovel_pampulha_2018_2019')
series_radiacao.to_csv('/content/drive/My Drive/Data Files/series_radiacao_pampulha_2018_2019')